In [1]:
import numpy as np

In [2]:
from gurobipy import Model,GRB

In [3]:
scm = Model("location-allocation-problem")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-08


In [8]:
p_cost_fixed = np.array([120,150,135])
p_cost_transport = np.array([[10,20,15,16],
                             [5,17,18,15],
                             [13,15,18,15]])
p_cap_max = np.array([540,510,600])
p_demand = np.array([15,35,22,41])
                             

In [ ]:
p_relation = [[0,1,1,0],
             [1,0,1,1],
             [0,0,0,1]]
p_relation_2 = {0:[1,2],1:[2,3],2:[0,1]}

In [ ]:
for i in range(n_prod):
    for j in range(n_dist):
        if p_relation[i,j] == 1:
            scm.addConstr( sum( v_transport[i,jp] for jp in range(n_dist)) <= 
                          p_cap_max[i]*v_locate[i])

In [ ]:
for p in p_relation_2:
    

In [11]:
n_prod = p_cost_transport.shape[0]
n_dist = p_cost_transport.shape[1]

In [12]:
# x_ij >= 0
v_transport = scm.addVars(n_prod,n_dist)
# y_i {0,1}
v_locate = scm.addVars(n_prod,vtype=GRB.BINARY)

In [13]:
for i in range(n_prod):
    scm.addConstr( sum( v_transport[i,j] for j in range(n_dist)) <= 
                  p_cap_max[i]*v_locate[i] )

In [14]:
for j in range(n_dist):
    scm.addConstr( sum(v_transport[i,j] for i in range(n_prod)) >=
                  p_demand[j])

In [17]:
scm.setObjective( sum(v_locate[i]*p_cost_fixed[i] for i in range(n_prod)) +
                      sum(v_transport[i,j]*p_cost_transport[i,j] for i in range(n_prod)
                          for j in range(n_dist)) , GRB.MINIMIZE)

In [18]:
scm.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i7-7500U CPU @ 2.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 7 rows, 15 columns and 27 nonzeros
Model fingerprint: 0x389cfadb
Variable types: 12 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [5e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 4e+01]
Presolve time: 0.03s
Presolved: 7 rows, 15 columns, 27 nonzeros
Variable types: 12 continuous, 3 integer (3 binary)
Found heuristic solution: objective 1950.0000000

Root relaxation: objective 1.679071e+03, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1679.07080    0    3 1950.00000 1679.07080  13.9%     -    0s
H 

In [19]:
scm.status

2

In [20]:
GRB.OPTIMAL

2

In [21]:
if scm.status == GRB.OPTIMAL:
    print(" objective is = ", np.round(scm.objVal,2))
    print(scm.getAttr('X',v_transport))
    print(scm.getAttr('X',v_locate))
else:
    print(" moodel cannot be solved! please check!")

 objective is =  1831.0
{(0, 0): 0.0, (0, 1): 0.0, (0, 2): 0.0, (0, 3): 0.0, (1, 0): 15.0, (1, 1): 35.0, (1, 2): 22.0, (1, 3): 41.0, (2, 0): 0.0, (2, 1): 0.0, (2, 2): 0.0, (2, 3): 0.0}
{0: 0.0, 1: 1.0, 2: 0.0}


In [24]:
xx = {s: v_transport[s].X for s in np.ndindex(n_prod,n_dist)}

In [27]:
import pandas as pd
df = pd.Series(xx)
df

0  0     0.0
   1     0.0
   2     0.0
   3     0.0
1  0    15.0
   1    35.0
   2    22.0
   3    41.0
2  0     0.0
   1     0.0
   2     0.0
   3     0.0
dtype: float64

# Matrix Programming

In [28]:
from gurobipy import Model,GRB

In [29]:
# classic
m = Model()
x1 = m.addVar(ub=1)
x2 = m.addVar(ub=1)
x3 = m.addVar(ub=1)
m.setObjective(x1*x1 + 2*x2*x2 + 3*x3*x3)
m.addConstr(x1 + 2*x2 + 3*x3 >= 4)
m.addConstr(x1 + x1 >= 1)
m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i7-7500U CPU @ 2.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 2 rows, 3 columns and 4 nonzeros
Model fingerprint: 0x786d793e
Model has 3 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 1 rows and 0 columns
Presolve time: 0.03s
Presolved: 1 rows, 3 columns, 3 nonzeros
Presolved model has 3 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   9.40303449e+05 -

In [31]:
# matrix prog
import numpy as np
m = Model()
Q = np.diag([1,2,3])
A = np.array([[1,2,3],
              [1,1,0]])
b = np.array([4,1])
x = m.addMVar(3,ub=1)
m.setObjective(x @ Q @ x)
m.addConstr(A @ x >= b)
m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i7-7500U CPU @ 2.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0xfe3c913b
Model has 3 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Presolve time: 0.05s
Presolved: 2 rows, 3 columns, 5 nonzeros
Presolved model has 3 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.000e+00
 Factor NZ  : 3.000e+00
 Factor Ops : 5.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   9.40049990e+05 -9.48045016e+05  9.99e+02 1.00e+03  1.0

In [30]:
np.diag([1,2,3])

array([[1, 0, 0],
       [0, 2, 0],
       [0, 0, 3]])